In [12]:
import numpy as np
import pandas as pd
from joblib import dump, load
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc("font", family='Microsoft YaHei')


# 加载数据
original_data = load('samples_data') 
# 将DataFrame转换为NumPy数组
original_data = original_data.values
print(original_data.shape)

(2330, 1025)


In [18]:
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import entropy
import nolds
from scipy.fft import fft, fftfreq
from scipy.signal import welch, find_peaks
from scipy.stats import skew

# 数据集制作
def make_dataset(data, split_rate = [0.7, 0.3]):
    '''
        参数:
        data   : 输入样本数据
        split_rate : 训练集、测试集划分比例

        返回:
        train_set: 训练集数据
        train_label: 训练集标签
        test_set: 测试集数据
        test_label: 测试集标签
    '''
    dataset = []
    for sample in data:
        features_lable = []
        # 提取信号
        signal = sample[0:1024]
        # 将NumPy数组转换为列表
        signal = signal.tolist()
        # 1. 峭度（Kurtosis）：衡量信号的尖锐程度，用于检测信号中的高频成分。
        kurt = kurtosis(signal)
        features_lable.append(kurt)
        # 2. 熵值（Entropy）：衡量信号的复杂程度和随机性，用于检测信号的频谱特性。
        ent = entropy(signal)#  -inf 表示  负无穷（-inf），则表示信号的熵为无穷大。这通常意味着信号具有非常高的不确定性和复杂性，其中的值没有明显的模式或规律
        if (ent<-0.000001 ):
            ent = 0
        features_lable.append(ent)
        # 3. 分形值（Fractal Dimension）：衡量信号的自相似性和复杂度，用于分析信号的分形特征。
        fd = nolds.dfa(signal)
        features_lable.append(fd)
        # 4. 波形指标（Waveform Indicators）：峰值因子，用于分析信号的时域特征。
        peak_factor = np.max(np.abs(signal)) / np.sqrt(np.mean(np.square(signal)))
        features_lable.append(peak_factor)
        # 5. 波形指标（Waveform Indicators）：脉冲因子，用于分析信号的时域特征。
        pulse_factor = np.max(np.abs(signal)) / np.mean(np.abs(signal))
        features_lable.append(pulse_factor)
        # 6. 波形指标（Waveform Indicators）：裕度因子，用于分析信号的时域特征。
        crest_factor = np.max(np.abs(signal)) / np.mean(np.sqrt(np.mean(np.square(signal))))
        features_lable.append(crest_factor)
        # 7. 频谱指标（Spectral Indicators）：能量比值，用于分析信号的频域特征。
        # 计算信号的频谱
        sampling_rate = 1024  # 采样率
        freq, power_spectrum = welch(signal, fs=sampling_rate)
        # 计算峰值频率
        peak_freqs, _ = find_peaks(power_spectrum, height=np.mean(power_spectrum))  # 找到峰值
        # 计算能量比值
        total_energy = np.sum(power_spectrum)
        peak_energy = np.sum(power_spectrum[peak_freqs])
        energy_ratio = peak_energy / total_energy
        features_lable.append(energy_ratio)
        # 8. 频谱指标（Spectral Indicators）：谱线形指标，用于分析信号的频域特征。
        # 计算谱线形指标
        spectral_flatness = np.exp(np.mean(np.log(power_spectrum))) / (np.mean(power_spectrum))
        features_lable.append(spectral_flatness)
        # 9. 统计特征（Statistical Features）：均值，用于描述信号的统计特性。
        mean = np.mean(signal)
        features_lable.append(mean)
        # 10. 统计特征（Statistical Features）：方差，用于描述信号的统计特性。
        variance = np.var(signal)
        features_lable.append(variance)
        # 11. 统计特征（Statistical Features）：偏度，用于描述信号的统计特性。
        skewness = skew(signal)
        features_lable.append(skewness)
        # 12. 振动特征（Vibration Features）：包括峰值振动、有效值振动等，用于描述信号的振动特性。
        peak_vibration = np.max(np.abs(signal))
        features_lable.append(peak_vibration)
        # 13. 振动特征（Vibration Features）：包括峰值振动、有效值振动等，用于描述信号的振动特性。
        rms_vibration = np.sqrt(np.mean(np.square(signal)))
        features_lable.append(rms_vibration)
        # 最后一步 添加标签
        features_lable.append(sample[-1])
        dataset.append(features_lable)

    # 将列表转换为NumPy数组
    dataset = np.array(dataset)

    # 划分数据集
    #序列数组
    sample_len = dataset.shape[0] # 样本总长度
    train_len = int(sample_len*split_rate[0])  # 向下取整
    # 变量数据 划分训练集、测试集
    train_set = dataset[:train_len, :13] # 训练集
    test_set = dataset[train_len:, :13]  # 测试集
    # y标签 划分训练集、测试集
    train_label = dataset[:train_len, -1] # 训练集
    test_label = dataset[train_len:, -1]  # 测试集

    return train_set, train_label, test_set, test_label


split_rate = [0.7, 0.3]

# 制作数据集
train_set, train_label, test_set, test_label = make_dataset(original_data, split_rate)
# 保存数据
dump(train_set, 'train_set') 
dump(train_label, 'train_label') 
dump(test_set, 'test_set') 
dump(test_label, 'test_label') 

['test_label']

In [19]:
print('数据 形状：')
print(train_set.shape, train_label.shape)
print(test_set.shape, test_label.shape)

数据 形状：
(1631, 13) (1631,)
(699, 13) (699,)
